In [1]:
class NODE:
    def __init__(self,key,parent):
        self.key = key
        self.value = 1
        self.children = None
        self.leafNode = True
        self.parent = parent
        
    def addEdge(self,childnode):
        self.leafNode = False
        if self.children is not None:
            self.children.append(childnode)
        else:
            self.children = []
            self.children.append(childnode)
    
    def increaseVal(self):
        self.value+=1
        
    def hasChild(self,key):
        if self.children is None:
            return None
        for i in self.children:
            if i.getKey() == key:
                return i
        return None
    
    def getKey(self):
        return self.key
    
    def getValue(self):
        return self.value
    
    def isLeaf(self):
        return self.leafNode
    
    def getParent(self):
        return self.parent
    
    def get_child_number(self):
        if self.children is None:
            return 0
        return len(self.children)
    
    def get_path(self,key=False):
        path = []
        current_node = self.getParent()
        while True:
            if current_node.getParent() is not None:
                if not key:
                    path.append(current_node)
                else:
                    path.append(current_node.getKey())
            else:
                break
            current_node = current_node.getParent()
        return (path[::-1],self.value)
    
    def get_single_path(self,key=False):
        path = []
        current_node = self
        while True:
            if current_node.getParent() is not None:
                if not key:
                    path.append(current_node)
                else:
                    path.append(current_node.getKey())
            else:
                break
            current_node = current_node.getParent()
        return path[::-1]
    
    def show(self):
        print("*"+str(self.key)+","+str(self.value)+"*")

In [2]:
import itertools as it

class TREE:
    def __init__(self,database,min_sup):
        self.root = NODE('null',None)
        self.nodes = []
        self.nodes.append(self.root)
        self.elements_count = []
        self.elements = []
        self.nodepointers = []
        self.min_sup = min_sup
        self.process_database(database)
    
    def min_path(self,path):
        count = 0
        min_value = 0
        for node in path:
            if count == 0:
                min_value = node.getValue()
                count+=1
            else:
                if node.getValue() < min_value:
                    min_value = node.getValue()
        return min_value
    
    def combination(self,l):
        li = []
        size = len(l)
        for i in range(1,size+1):
            temp_li = list(it.combinations(l,i))
            temp_li = [list(x) for x in temp_li]
            li.extend(temp_li)
        return li
    
    def process_database(self,database):
        self.processed_database = []
        for row in database:
            for element in row:
                if element not in self.elements:
                    self.elements.append(element)
                    num = self.has(element,database)
                    if num >= self.min_sup:
                        self.elements_count.append((element,num))
        self.elements_count.sort(key=lambda x: (-x[1],x[0]))
        self.selected_elements = [x[0] for x in self.elements_count]
        
        for row in database:
            new_row = []
            for element in self.selected_elements:
                if element in row:
                    new_row.append(element)
            self.processed_database.append(new_row)
        self.add_database(self.processed_database)
   
    def has(self,element,database):
        count = 0
        for i in database:
            if element in set(i):
                count += 1
        return count
    
    def add_database(self,database):
        self.nodepointers = [[] for i in self.selected_elements]
        for i in database:
#            print i
            self.add_transaction(i)
        
    def add_transaction(self,transaction):
        current_node = self.root
        for j in transaction:
            child = current_node.hasChild(j)
            if child is None:    
                new_node = NODE(j,current_node)
                self.nodes.append(new_node)
                current_node.addEdge(new_node)
                current_node = new_node
                self.nodepointers[self.selected_elements.index(new_node.getKey())].append(new_node)
            else:
                child.increaseVal()
                current_node= child
                
    def get_elements(self):
        return self.elements
    
    def get_root(self):
        return self.root
    
    def get_leaf_nodes(self):
        leaf_nodes = []
        for node in self.nodes:
            if node.isLeaf():
                leaf_nodes.append(node)
        return leaf_nodes
    
    def get_paths(self,key):
        paths = []
        nodes = self.nodepointers[self.selected_elements.index(key)]
        for node in nodes:
            path = node.get_path()
            if len(path[0])!=0:
                paths.append(path)
                
        if len(paths)==0:
            return None
        return paths
    
    def get_processed_database(self):
        return self.processed_database
    
    def show(self):
        for node in self.nodes:
            node.show()
            
    def single_path(self):
        current = self.root
        while True:
            if current.get_child_number() > 1:
                return False
            elif current.get_child_number() == 0:
                return True
            elif current.get_child_number() == 1:
                current = current.children[0]
                
    def get_single_path(self):
        try:
            node = self.nodepointers[-1][0]
            return node.get_single_path()
        except:
            self.show()

In [5]:
import time
            
def get_conditional_fptree(tree,key):
    paths = tree.get_paths(key)
    if paths is None:
        return None
    database = []
    for path in paths:
        for i in range(path[1]):
            path_key = [x.getKey() for x in path[0]]
            database.append(path_key)
    conditional_fptree = TREE(database,tree.min_sup)
    return conditional_fptree

def generate_pattern(tree,alpha=None):
    patterns = []
    if tree is None or len(tree.nodes)==1:
        return [alpha]
    if tree.single_path():
        path = tree.get_single_path()
        all_patterns = tree.combination(path)
        for pattern in all_patterns:
            if tree.min_path(pattern) >= tree.min_sup:
                pattern = [x.getKey() for x in pattern]
                if alpha is not None:
                    pattern.extend([x for x in alpha])
                patterns.append(pattern)
    else:
        selected_inv = tree.selected_elements[::-1]
        for element in selected_inv:
            beta = [element]
            if alpha is not None:
                for i in alpha:
                    beta.append(i)
            patterns.append(beta)
            patterns.extend(generate_pattern(get_conditional_fptree(tree,element),alpha=beta))
    return patterns

start = time.time()
filename = "mushroom.txt"
with open(filename,'r') as fobj:
    Database = [[int(num) for num in line.split()] for line in fobj]
#Database = [[1,2,5],[1,2,4],[2,3],[2,4],[1,3],[2,3],[1,3],[1,2,3,5],[1,2,3]]
# print(Database)
min_sup_per = 0.2#input("Input minimum support percentage: ")
min_sup = (min_sup_per * len(Database))

fptree = TREE(Database,min_sup)
elements = fptree.selected_elements[::-1]

    
patterns = generate_pattern(fptree)
patterns = set(map(tuple,patterns))
patterns = list(map(list,patterns))
patterns.sort(key=lambda x:len(x))
current = 1
count = 0
total = 0
print("L"+str(current)+": ",end= '')
for pattern in patterns:
    if len(pattern) != current:
        current+=1
        print(count)
        total+=count
        count = 0
        print("L"+str(current)+": ",end= '')
    count+=1
total+=count
print("Number of total itemsets: "+str(total))
print(time.time()-start)

L1: 43
L2: 376
L3: 1472
L4: 3559
L5: 6267
L6: 8802
L7: 10151
L8: 9488
L9: 7010
L10: 4004
L11: 1729
L12: 546
L13: 119
L14: 16
L15: Number of total itemsets: 53583
19.470642566680908
